In [3]:
import datetime
import numpy as np
import pandas as pd
import vectorbt as vbt
import plotly.io as pio
import plotly.graph_objects as go

# Este notebook exemplifica a otimização de parâmetros afim de maximizar o lucro do portfólio.

In [4]:
end_time = datetime.datetime.now()

start_time = end_time - datetime.timedelta(days=365)

In [5]:
list_stock_enterprises = [
    'RRRP3.SA', 
    'CSAN3.SA', 
    'DMMO3.SA', 
    'ENAT3.SA', 
    'PRIO3.SA',
    'PETR3.SA',
    'RECV3.SA',
    'RPMG3.SA',
    'UGPA3.SA',
    'VBBR3.SA',

    ]

In [6]:
b3_petroleum = vbt.YFData.download(
    list_stock_enterprises,
    missing_index="drop",
    interval = "1d",
    start = start_time,
    end = end_time
)

stock_prices = b3_petroleum.get("Close")

c:\Users\joaop\Anaconda3\envs\vbt\lib\site-packages\vectorbt\data\base.py:692: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  new_data[c].loc[:, s] = col_data
c:\Users\joaop\Anaconda3\envs\vbt\lib\site-packages\vectorbt\data\base.py:692: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  new_data[c].loc[:, s] = col_data
c:\Users\joaop\Anaconda3\envs\vbt\lib\site-packages\vectorbt\data\base.py:692: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old beh

In [7]:
windows = np.arange(2, 101)
fast_ma, slow_ma = vbt.MA.run_combs(stock_prices, window=windows, r=2, short_names=['fast', 'slow'])
entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

pf_kwargs = dict(size=np.inf, fees=0.001, freq='1D')
pf = vbt.Portfolio.from_signals(stock_prices, entries, exits, **pf_kwargs)

returns = pf.total_return()

In [8]:
print(returns.max())
print(returns.idxmax()) 

print(pf[returns.idxmax()].stats())

pf[returns.idxmax()].plot().show()

1.5983042576734052
(21, 37, 'DMMO3.SA')
Start                         2021-10-18 03:00:00+00:00
End                           2022-10-17 03:00:00+00:00
Period                                250 days 00:00:00
Start Value                                       100.0
End Value                                    259.830426
Total Return [%]                             159.830426
Benchmark Return [%]                         140.540534
Max Gross Exposure [%]                            100.0
Total Fees Paid                                0.893831
Max Drawdown [%]                              38.172045
Max Drawdown Duration                  72 days 00:00:00
Total Trades                                          3
Total Closed Trades                                   2
Total Open Trades                                     1
Open Trade PnL                                46.497959
Win Rate [%]                                      100.0
Best Trade [%]                                83.715999
Worst Tr